## Steps

All steps that are played by default are **preprocessing** steps, but you can also add **postprocessing** steps, that act on the results instead of the circuit.

The steps will be filtered according to their type automatically.

In [4]:
from pennylane_calculquebec.processing.config import MonarqDefaultConfig
from pennylane_calculquebec.processing.steps import ReadoutErrorMitigation

readout_error_mitigation = ReadoutErrorMitigation()

my_config = MonarqDefaultConfig()
my_config.steps.append(readout_error_mitigation)

You can create new preprocessing / postprocessing steps by overriding the PreProcessing / PostProcessing classes.

In [5]:
# abstract steps + empty config
from pennylane_calculquebec.processing.interfaces import PreProcStep, PostProcStep
from pennylane_calculquebec.processing.config import ProcessingConfig

# default steps
from pennylane_calculquebec.processing.steps import CliffordTDecomposition, ASTAR, Swaps, IterativeCommuteAndMerge, MonarqDecomposition

In [35]:
# toy preprocessing step for printing the circuit operations
class PrintCircuit(PreProcStep):
    def execute(self, tape):
        print(*tape.operations)
        return tape
        
# toy postprocessing step for printing the results
class PrintResults(PostProcStep):
    def execute(self, tape, results):
        print(results)
        return results

In [36]:
# this custom config will print the circuit, transpile, print the transpiled circuit 
# and then print the unmitigated results, followed by the mitigated results.
my_config = ProcessingConfig(PrintCircuit(),
                             CliffordTDecomposition(), 
                             ASTAR(),
                             Swaps(),
                             IterativeCommuteAndMerge(),
                             MonarqDecomposition(), 
                             PrintCircuit(),
                             PrintResults(),
                             ReadoutErrorMitigation(), 
                             PrintResults())


### let's try our config with custom steps

In [ ]:
import pennylane as qml
from pennylane_calculquebec.API.client import MonarqClient

# Change the values in the parentheses for your credentials
my_client = MonarqClient("your host", "your user", "your access token", "your project")

dev = qml.device("monarq.default", client=my_client, processing_config = my_config, shots=1000)

# a simple ghz circuit
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1]) 
    qml.CNOT([1, 2])
    return qml.counts()

# lets print the circuit
print(qml.draw(circuit)())

results = circuit()

# you don't have to print results, since they are printed as a post processing step!
# print(results)

0: ──H─╭●────┤  Counts
1: ────╰X─╭●─┤  Counts
2: ───────╰X─┤  Counts
H(0) CNOT(wires=[0, 1]) CNOT(wires=[1, 2])
X90(wires=[1]) X90(wires=[4]) Z(4) CZ(wires=[1, 4]) X90(wires=[4]) X90(wires=[0]) Z(0) CZ(wires=[4, 0]) X90(wires=[0])
{'000': 376, '001': 38, '010': 44, '011': 58, '100': 126, '101': 65, '110': 98, '111': 195}
{'000': 418.0, '001': 16.0, '010': 12.0, '011': 61.0, '100': 87.0, '101': 20.0, '110': 71.0, '111': 315.0}
